# STEP 1: Classificatore OCCLUSION vs VISIBLE

Classificatore binario per distinguere immagini occluse da immagini visibili.

**Label mapping:**
- `OCCLUSION` / `PARTIAL OCCLUSION` → classe 0
- `OK` / `KO` → classe 1 (VISIBLE)


In [ ]:
# Setup: Clona repository GitHub e monta Google Drive per i dati
import os
from pathlib import Path

# Opzione 1: Clona da GitHub (consigliato per sviluppo)
# Sostituisci con il tuo repository URL
GITHUB_REPO = "https://github.com/Giovanni000/Project-Work.git"  # ⚠️ MODIFICA QUESTO!
REPO_DIR = "/content/project"

# Clona repository (se non esiste già)
if not Path(REPO_DIR).exists():
    !git clone {GITHUB_REPO} {REPO_DIR}

# Cambia directory al repository
os.chdir(REPO_DIR)
print(f"Repository directory: {os.getcwd()}")

# Opzione 2: Monta Google Drive solo per i dati (immagini)
from google.colab import drive
drive.mount('/content/drive')

# Path ai dati su Drive
DATA_ROOT = Path("/content/drive/MyDrive/Project Work/Data")
print(f"Data directory: {DATA_ROOT}")

# Crea symlink o copia i dati se necessario
# Oppure modifica i path nel CSV per puntare a Drive

# Import necessari
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import pandas as pd
from pathlib import Path
import numpy as np
from tqdm import tqdm

# Seed per riproducibilità
torch.manual_seed(42)
np.random.seed(42)

# Verifica device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")


## Dataset Class


In [ ]:
class OcclusionDataset(Dataset):
    """
    Dataset PyTorch per classificazione OCCLUSION vs VISIBLE.
    
    Label mapping:
    - OCCLUSION (o PARTIAL OCCLUSION) → 0
    - OK o KO → 1 (VISIBLE)
    """
    
    def __init__(self, csv_path, transform=None):
        """
        Args:
            csv_path: Path al CSV con colonne 'image_path' e 'label'
            transform: Trasformazioni da applicare alle immagini
        """
        self.df = pd.read_csv(csv_path)
        self.transform = transform
        
        # Mappa label: OCCLUSION → 0, OK/KO → 1
        self.label_map = {
            'OCCLUSION': 0,
            'OK': 1,
            'KO': 1
        }
        
        print(f"Dataset caricato: {len(self.df)} immagini")
        print(f"Distribuzione label originale:")
        print(self.df['label'].value_counts())
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        label_str = row['label']
        
        # Carica immagine
        image = Image.open(image_path).convert('RGB')
        
        # Applica trasformazioni
        if self.transform:
            image = self.transform(image)
        
        # Converti label
        label = self.label_map.get(label_str, 1)  # Default a VISIBLE se label sconosciuta
        
        return image, label


## Modello CNN


In [ ]:
class OcclusionCNN(nn.Module):
    """
    CNN semplice per classificazione binaria OCCLUSION vs VISIBLE.
    
    Architettura:
    - 3 layer convoluzionali con pooling
    - 2 layer fully connected
    - Output: 2 classi (OCCLUSION, VISIBLE)
    """
    
    def __init__(self):
        super(OcclusionCNN, self).__init__()
        
        # Encoder convoluzionale
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # 128x128 -> 64x64
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)  # 64x64 -> 32x32
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)  # 32x32 -> 16x16
        
        # Fully connected
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 2)  # 2 classi: OCCLUSION, VISIBLE
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # Convoluzioni
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu(self.bn3(self.conv3(x))))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


## Training Loop


In [ ]:
def train_occlusion_classifier(csv_path="data/dataset.csv", 
                               val_fraction=0.2,
                               batch_size=32,
                               num_epochs=20,
                               learning_rate=0.001,
                               device=None):
    """
    Training loop per il classificatore OCCLUSION vs VISIBLE.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"Device: {device}")
    print(f"PyTorch version: {torch.__version__}")
    
    # Trasformazioni per le immagini
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
    ])
    
    # Crea dataset
    full_dataset = OcclusionDataset(csv_path, transform=transform)
    
    # Split train/val
    total_size = len(full_dataset)
    val_size = int(val_fraction * total_size)
    train_size = total_size - val_size
    
    train_dataset, val_dataset = random_split(
        full_dataset, 
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    print(f"\nSplit dataset:")
    print(f"  Train: {len(train_dataset)} immagini")
    print(f"  Val: {len(val_dataset)} immagini")
    
    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    # Modello
    model = OcclusionCNN().to(device)
    print(f"\nModello creato. Parametri totali: {sum(p.numel() for p in model.parameters()):,}")
    
    # Loss e optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    best_val_acc = 0.0
    best_model_state = None
    
    print(f"\nInizio training per {num_epochs} epoch...")
    print("-" * 60)
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]"):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        
        train_acc = 100 * train_correct / train_total
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)
        
        # Stampa risultati
        print(f"Epoch {epoch+1}/{num_epochs}:")
        print(f"  Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        # Salva miglior modello
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
            print(f"  ✓ Nuovo miglior modello! Val Acc: {best_val_acc:.2f}%")
        
        print("-" * 60)
    
    # Carica miglior modello
    model.load_state_dict(best_model_state)
    print(f"\n✅ Training completato!")
    print(f"   Miglior Val Accuracy: {best_val_acc:.2f}%")
    
    # Salva modello
    models_dir = Path("models")
    models_dir.mkdir(exist_ok=True)
    model_path = models_dir / "occlusion_cnn.pth"
    torch.save(model.state_dict(), model_path)
    print(f"   Modello salvato in: {model_path}")
    
    return model


In [ ]:
# Training del classificatore
model_occ = train_occlusion_classifier(
    csv_path="data/dataset.csv",
    val_fraction=0.2,
    batch_size=32,
    num_epochs=20,
    learning_rate=0.001,
    device=device
)


## Funzione Helper per Caricare il Modello


In [ ]:
def load_occlusion_model(device=None):
    """
    Carica il modello addestrato per classificazione OCCLUSION vs VISIBLE.
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model = OcclusionCNN().to(device)
    model_path = Path("models/occlusion_cnn.pth")
    
    if not model_path.exists():
        raise FileNotFoundError(f"Modello non trovato: {model_path}")
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    return model
